# Random Forest & Adaboost

data

In [1]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd
import random
random.seed(2021)
import pickle

In [2]:
path = "C:/Users/liqia/Spring2021-Project3-group-3-master/data//train_set"
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

In [3]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [4]:
read_time_start=time.time()
X, Y = read_data(path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))
X.shape, Y.shape 

Read the original dataset takes 107.238 seconds


((3000, 3003), (3000,))

In [5]:
print('majority class: %d' % np.sum(Y == 0))
print('minority class: %d' % np.sum(Y == 1))
#imbalanced dataset

majority class: 2402
minority class: 598


# oversample then split data

In [6]:
def data_preprocessing(X, Y, path):

  distances = X
  y = Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [7]:
X_balanced, Y_balanced = data_preprocessing(X, Y, path)
X_balanced.shape, Y_balanced.shape

((4804, 3003), (4804,))

In [8]:
print('majority class: %d' % np.sum(Y_balanced == 0))
print('minority class: %d' % np.sum(Y_balanced == 1))
#balanced dataset

majority class: 2402
minority class: 2402


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd

In [10]:
from sklearn.model_selection import train_test_split
X_balanced_train,X_balanced_test,y_balanced_train,y_balanced_test= train_test_split(X_balanced, Y_balanced,test_size=0.2,random_state=0)
X_balanced_train.shape,X_balanced_test.shape,y_balanced_train.shape,y_balanced_test.shape

((3843, 3003), (961, 3003), (3843,), (961,))

In [11]:
print('majority train class: %d' % np.sum(y_balanced_train == 0))
print('minority train class: %d' % np.sum(y_balanced_train == 1))
print('majority test class: %d' % np.sum(y_balanced_test == 0))
print('minority test class: %d' % np.sum(y_balanced_test == 1))

majority train class: 1918
minority train class: 1925
majority test class: 484
minority test class: 477


# split then oversample

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.20,random_state=0)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print('majority train class: %d' % np.sum(Y_train == 0))
print('minority train class: %d' % np.sum(Y_train == 1))
print('majority test class: %d' % np.sum(Y_test == 0))
print('minority test class: %d' % np.sum(Y_test == 1))
#imbalanced dataset

majority train class: 1923
minority train class: 477
majority test class: 479
minority test class: 121


In [13]:
def data_preprocessing(X_train, Y_train, path):

  distances = X_train
  y = Y_train

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [14]:
X_train_balanced, y_train_balanced = data_preprocessing(X_train, Y_train, path)
X_train_balanced.shape, y_train_balanced.shape

((3846, 3003), (3846,))

In [15]:
def data_preprocessing(X_test, Y_test, path):

  distances = X_test
  y = Y_test

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [16]:
X_test_balanced, y_test_balanced = data_preprocessing(X_test, Y_test, path)
X_test_balanced.shape, y_test_balanced.shape
print('majority train class: %d' % np.sum(y_train_balanced == 0))
print('minority train class: %d' % np.sum(y_train_balanced == 1))
print('majority test class: %d' % np.sum(y_test_balanced == 0))
print('minority test class: %d' % np.sum(y_test_balanced == 1))

majority train class: 1923
minority train class: 1923
majority test class: 479
minority test class: 479


# divide into 2 chunk 3 groups 

        
    • balanced train
    
        1.(train with balanced,test with balanced)
           a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
           b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
        
        2.(train with balanced,test with imbalanced) 
            X_train_balanced.y_train_balanced;X_test,Y_test
      

    • imbalanced train
    
        3. (train with imbalanced,test with imbalanced)
            X_train,Y_train; X_test,Y_test 

model

In [17]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 93.865 seconds
Predicting test data takes 1.503 seconds
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       484
           1       0.87      0.86      0.86       477

    accuracy                           0.86       961
   macro avg       0.86      0.86      0.86       961
weighted avg       0.86      0.86      0.86       961



In [18]:
#Save RandomForest model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest1a.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest1a.p",'rb'))

RandomForestClassifier(class_weight='balanced', n_estimators=300)

In [19]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 111.639 seconds
Predicting test data takes 0.332 seconds
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       479
           1       0.78      0.70      0.73       479

    accuracy                           0.75       958
   macro avg       0.75      0.75      0.75       958
weighted avg       0.75      0.75      0.75       958



In [20]:
#Save RandomForest model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest1b.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest1b.p",'rb'))

RandomForestClassifier(class_weight='balanced', n_estimators=300)

In [21]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 112.358 seconds
Predicting test data takes 0.245 seconds
              precision    recall  f1-score   support

           0       0.85      0.80      0.82       479
           1       0.35      0.43      0.39       121

    accuracy                           0.72       600
   macro avg       0.60      0.61      0.60       600
weighted avg       0.75      0.72      0.73       600



In [22]:
#Save RandomForest model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest2.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest2.p",'rb'))

RandomForestClassifier(class_weight='balanced', n_estimators=300)

In [23]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 37.175 seconds
Predicting test data takes 0.169 seconds
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       479
           1       0.71      0.10      0.17       121

    accuracy                           0.81       600
   macro avg       0.76      0.54      0.53       600
weighted avg       0.79      0.81      0.75       600



In [24]:
#Save RandomForest model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest3.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/RandomForest3.p",'rb'))

RandomForestClassifier(class_weight='balanced', n_estimators=300)

In [ ]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

In [ ]:
#Save AdaBoost model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1a.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1a.p",'rb'))

In [26]:
#1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 352.041 seconds
Predicting test data takes 718.837 seconds
              precision    recall  f1-score   support

           0       0.76      0.75      0.75       479
           1       0.75      0.76      0.75       479

    accuracy                           0.75       958
   macro avg       0.75      0.75      0.75       958
weighted avg       0.75      0.75      0.75       958



In [28]:
#Save AdaBoost model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1b.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1b.p",'rb'))

AdaBoostClassifier(n_estimators=200)

In [ ]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

In [ ]:
#Save AdaBoost model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost2.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost2.p",'rb'))

In [ ]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

In [ ]:
#Save AdaBoost model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost3.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost3.p",'rb'))

In [ ]:
#1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

In [ ]:
#Save AdaBoost100 model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1001a.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1001a.p",'rb'))

In [ ]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

In [ ]:
#Save AdaBoost100 model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1001b.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1001b.p",'rb'))

In [ ]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

In [ ]:
#Save AdaBoost100 model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1002.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1002.p",'rb'))

In [ ]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_train, Y_train)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

In [ ]:
#Save AdaBoost100 model
pickle.dump(clf, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1003.p",'wb'))


#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/AdaBoost1003.p",'rb'))